# An Introduction to the Gaussian Distribution Class

This tutorial illustrates how to use `probpipe.distributions.gaussian.Gaussian`, proppipe's implementation of a multivariate
Gaussian distribution.


In [1]:
import numpy as np

from probpipe.distributions.real_vector import Gaussian
from probpipe.linalg.linear_operator import DenseLinOp, DiagonalLinOp, TriangularLinOp, CholeskyLinOp

In [2]:
# Helper to test equality of arrays
def approx_equal(a, b):
    return np.allclose(a, b, rtol=0, atol=1e-10)

## The Basics
We begin by constructing a bivariate Gaussian distribution.

In [3]:
m = np.array([0, 1])
C = np.array([[1.0, 0.8], [0.8, 1.0]])
x = Gaussian(m, C)

print(x)

The `sample()` and `log_density()` methods are used to draw samples and evaluate the log probability
density function, respectively. A set of `n` samples is returned as a `(n, d)` array, with `d` the 
dimension of the distribution. The `log_density()` method is vectorized to compute the log density
over a batch of input points, stored as an array of shape `(n, d)` or `(d,)`, the latter corresponding
to a single point. Notice that a single sample is returned as an array of shape `(1, d)` for consistency
with the batch convention. Other methods such as `density()` and `cdf()` follow similar batch conventions.

In [4]:
# Draw samples
samp_batch = x.sample(5)
single_sample = x.sample()

# Evaluate log density
log_dens_batch = x.log_density(samp_batch)
log_dens_single_sample = x.log_density(single_sample)

print("Batch of samples:")
print(samp_batch)

print("\nSingle sample:")
print(single_sample)

print("\nLog density at batch of samples:")
print(log_dens_batch)

print("\nLog density at single sample:")
print(log_dens_single_sample)


Batch of samples:
[[-1.27084369 -1.29391864]
 [ 0.85565143  1.97327043]
 [-0.17241531  0.77292426]
 [-1.14172371  0.11251466]
 [-1.28106545 -0.01694943]]

Single sample:
[[1.02021509 1.08260283]]

Log density at batch of samples:
[-4.40033916 -1.80892134 -1.35295185 -1.97974918 -2.14770253]

Log density at single sample:
[-2.59486509]


The mean and covariance of the Gaussian distribution can be accessed as follows. These properties
are also defined for any subclass of `RealVectorDistribution`.

In [5]:
print(x.mean)
print(x.cov)

[0 1]
DenseLinOp(shape=(2, 2), dtype=float64)


Observe that the mean is a flat array of length `d`, as expected. On the other hand, the covariance is
a `DenseLinOp` instance. A linear operator is simply an abstract representation of a matrix. A dense 
linear operator actually stores the full matrix as a dense array. The concrete array representation of
any linear operator can be obtained via the `to_dense()` method. Note that the `Gaussian` constructor
accepted the array representation of the covariance, but this was wrapped as a `DenseLinOp` under 
the hood.

In [6]:
cov_op = x.cov
cov_op.to_dense()

array([[1. , 0.8],
       [0.8, 1. ]])

## Structured Covariance Operators

The benefit of representing the covariance as a linear operator is that we can pass linear 
operators representing structured (sparse) matrices. This allows the user to work with the 
covariance as they typically would with a concrete matrix, but under the hood linear algebra
operations are performed that exploit the structure in the matrix. We walk through a few 
different structured covariance operators below. 

### Diagonal Covariance

We start by considering a diagonal covariance matrix. In this case, we need only store `d` numbers, the 
variances forming the diagonal of the matrix. The full `(d, d)` matrix can always be formed by calling 
`to_dense()`, but this is rarely necessary nor recommended. Operations involving the diagonal linear 
operator (e.g., those necessary to sample from the Gaussian or compute its log density) will be carried 
out efficiently using only the diagonal entries.

In [7]:
cov_diag = DiagonalLinOp(np.array([1., 4., 16.]))

print(cov_diag)
print(cov_diag.shape)
print(cov_diag.dtype)
print(cov_diag.diagonal)
print(cov_diag.to_dense())

DiagonalLinOp(shape=(3, 3), dtype=float64)
(3, 3)
float64
[ 1.  4. 16.]
[[ 1.  0.  0.]
 [ 0.  4.  0.]
 [ 0.  0. 16.]]


In [8]:
x = Gaussian(np.zeros(3), cov_diag)
samp = x.sample(3)
dens = x.density(samp)


print("The covariance is represented by the diagonal linear operator:")
print(x.cov)

print("\nThe sampling method will exploit the sparse structure of the operator:")
print(samp)

print("\nAs will the density and other methods:")
print(dens)

The covariance is represented by the diagonal linear operator:
DiagonalLinOp(shape=(3, 3), dtype=float64)

The sampling method will exploit the sparse structure of the operator:
[[ 0.43859308 -0.84378776  2.66143745]
 [-0.64126997 -2.1440897   1.06900133]
 [-0.97223546  0.42106828 -1.77114245]]

As will the density and other methods:
[0.0052855  0.00350971 0.00438717]


In [9]:
# Confirm equivalence with dense representation.
x_dense = Gaussian(x.mean, cov_diag.to_dense())
print(x_dense.cov)

print("\nLog density evaluations are equal for dense vs. diagonal representation:")
print(approx_equal(x.log_density(samp),
                   x_dense.log_density(samp)))

DenseLinOp(shape=(3, 3), dtype=float64)

Log density evaluations are equal for dense vs. diagonal representation:
True


### Covariance Represented by Cholesky Factor

In algorithms involving Gaussian distributions, it is common to represent the Gaussian covariance 
$C$ indirectly via its Cholesky factor. If using the lower Cholesky factor this is a `(d, d)` lower
triangular matrix $L$ satisfying $C = LL^\top$. We can construct a Gaussian from its Cholesky factor
by passing the covariance as a `CholeskyLinOp` object. We first represent `L` itself as a `TriangularLinOp`
before feeding this to `CholeskyLinOp`.

We'll also construct a Gaussian with a dense covariance operator for comparison.

In [10]:
# Mean and covariance of Gaussian
m = np.array([0, 1])
C = np.array([[1.0, 0.8], [0.8, 1.0]])

# Dense representation
x_dense = Gaussian(m, C)

# Cholesky factor
L = np.linalg.cholesky(C, upper=False)
print("The Cholesky factor:")
print(L)

# Cholesky representation of covariance
L = TriangularLinOp(L, lower=True)
C_from_L = CholeskyLinOp(L)
x_chol = Gaussian(m, C_from_L)


The Cholesky factor:
[[1.  0. ]
 [0.8 0.6]]


In [11]:
# Investigating the covariance operator

print("Covariance operator:")
print(x_chol.cov)

print("\nThe Cholesky root (also represented as a linear operator):")
print(x_chol.cov.root)

print("\nDense Cholesky factor:")
print(x_chol.cov.root.to_dense())

print("\nDense covariance matrix:")
print(x_chol.cov.to_dense())

Covariance operator:
CholeskyLinOp(shape=(2, 2), dtype=float64)

The Cholesky root (also represented as a linear operator):
TriangularLinOp(shape=(2, 2), dtype=float64)

Dense Cholesky factor:
[[1.  0. ]
 [0.8 0.6]]

Dense covariance matrix:
[[1.  0.8]
 [0.8 1. ]]


In [12]:
# Testing that the two representations are equivalent
samp = x_chol.sample(10)

print("Log density evaluations of x_chol and x_dense are equal:")
print(approx_equal(x_chol.log_density(samp),
                   x_dense.log_density(samp)))

Log density evaluations of x_chol and x_dense are equal:
True


### A Peak Under the Hood

At present, when a `Gaussian` instance is initialized, the `cov` argument is used to compute the 
lower Cholesky factor of the covariance matrix, which is then stored and used for subsequent 
computations. This Cholesky factor is itself stored as a linear operator object in the 
`_cov_from_chol` attribute. The original operator passed to the constructor is still 
available via the `cov` property. In general, `_cov_from_chol` will be a `CholeskyLinOp` instance,
but it may be a more structured operator in special cases. For example, if the user passes 
`cov` as a `DiagonalLinOp` then `_cov_from_chol` will be a `DiagonalRootLinOp` instance.

It should be emphasized that `cov` and `_cov_from_chol` represent the same exact covariance
matrix. They simply provide different representations of the matrix. Caching the lower triangular 
factor `_cov_from_chol.root` allows for efficient computations, which especially pays off when 
many operations are to be performed with a fixed Gaussian distribution. 

In [13]:
# The Gaussian distribution called to_cholesky_representation() to obtain the lower Cholesky factor
C_dense = DenseLinOp([[1., 0.8], [0.8, 1.]])
C_from_chol = C_dense.to_cholesky_representation()

print(C_from_chol)
print(C_from_chol.root.to_dense())
print(C_from_chol.to_dense())

CholeskyLinOp(shape=(2, 2), dtype=float64)
[[1.  0. ]
 [0.8 0.6]]
[[1.  0.8]
 [0.8 1. ]]


In [14]:
# to_cholesky_representation() exploits the diagonal structure
C_diag = DiagonalLinOp([1., 4.])
C_from_chol = C_diag.to_cholesky_representation()

print(C_from_chol)
print(C_from_chol.root.to_dense())
print(C_from_chol.to_dense())

DiagonalRootLinOp(shape=(2, 2), dtype=float64)
[[1. 0.]
 [0. 2.]]
[[1. 0.]
 [0. 4.]]


In [15]:
# We can observe this behavior in the Gaussian constructor
x_dense = Gaussian(np.zeros(2), C_dense)

print(x_dense.cov)
print(x_dense._cov_from_chol)

DenseLinOp(shape=(2, 2), dtype=float64)
CholeskyLinOp(shape=(2, 2), dtype=float64)


In [16]:
# And same with the diagonal case
x_diag = Gaussian(np.zeros(2), C_diag)

print(x_diag.cov)
print(x_diag._cov_from_chol)

DiagonalLinOp(shape=(2, 2), dtype=float64)
DiagonalRootLinOp(shape=(2, 2), dtype=float64)
